In [2]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# import getpass

In [2]:
url = 'https://www.instagram.com/gyeonggi_korea/'

In [3]:
# webdriver 로드
driver = webdriver.Chrome('chromedriver')
driver.get(url)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title

<title>gyeonggi_korea(경기도)(@gyeonggi_korea) • Instagram 사진 및 동영상</title>

# 팔로워 수 크롤링

In [4]:
# 글 내용 크롤링
page = soup.find_all('li', class_='Y8-fY')
txt = page[1].text[4:]
txt = txt.replace(',', '')
txt = int(txt)
txt

[<li class="Y8-fY"><a href="/accounts/login/?next=%2Fgyeonggi_korea%2F&amp;source=profile_posts" tabindex="0"><div class="_7UhW9 vy6Bb MMzan KV-D4 uL8Hv T0kll">게시물 <span class="g47SY">774</span></div></a></li>,
 <li class="Y8-fY"><a href="/accounts/login/?next=%2Fgyeonggi_korea%2Ffollowers%2F&amp;source=followed_by_list" tabindex="0"><div class="_7UhW9 vy6Bb MMzan KV-D4 uL8Hv T0kll">팔로워 <span class="g47SY" title="9,771">9,771</span></div></a></li>,
 <li class="Y8-fY"><a href="/accounts/login/?next=%2Fgyeonggi_korea%2Ffollowing%2F&amp;source=follows_list" tabindex="0"><div class="_7UhW9 vy6Bb MMzan KV-D4 uL8Hv T0kll">팔로우 <span class="g47SY">3,329</span></div></a></li>]

In [3]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.instagram.com/accounts/login/")

In [4]:
# username = getpass.getpass("Input ID : ") # User ID
# password = getpass.getpass("Input PWD : ") # User PWD, 특히 getpass를 통해서 비밀번호 정보를 숨김. 
 
# element_id = driver.find_element_by_name("username")
# element_id.send_keys(username)
# element_password = driver.find_element_by_name("password")
# element_password.send_keys(password)

username = 'gyeonggi_korea'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
element_id.send_keys(username)
element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
element_password.send_keys(password)

time.sleep(1.5)

<ipython-input-4-a45129e72321>:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
<ipython-input-4-a45129e72321>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')


In [5]:
##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

<ipython-input-5-31e51f34e6fc>:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()


In [6]:
url = 'https://www.instagram.com/gyeonggi_korea/'
# webdriver 로드
driver.get(url)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title

<title>gyeonggi_korea(경기도)(@gyeonggi_korea) • Instagram 사진 및 동영상</title>

In [18]:
post_link = []
div = soup.find_all('div', class_="Nnq7C weEfm")

# 게시글 줄 순서(1번째부터 몇 번째 줄까지 가져올 것인지)
number_of_line = 2
# 게시글 링크 3개씩 가져오기
for row in range(number_of_line):
    a_tags = div[row].find_all('a')
    for i in range(3):
        link = a_tags[i].attrs['href']
        post_link.append(link)
# 수집한 링크 개수 확인
print("수집한 링크의 개수 : ", len(post_link))
print("수집한 링크 주소")
print(post_link)

수집한 링크의 개수 :  6
수집한 링크 주소
['/p/CamCGL3LSHX/', '/p/Cag862TrWuX/', '/p/CaWWan2BoZc/', '/p/CaRgX_svAjY/', '/p/CaG5pIbv7MV/', '/p/CZ9rGIHP423/']


In [14]:
len(a_tags)

3

In [24]:
# post_link = []
# for postline in soup.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
#        a_len = len(postline.select('a'))
        # 인스타그램 게시물은 행별로 최대 3개까지 확인할 수 있는데, 최근게시물이나 마지막 게시물은 1,2개가 나올 수도 있어서 len 지정
#        for post in range(a_len):   
#            item = postline.select('a')[post]
#            link = item.attrs['href']
#            post_link.append(link)
# post_link

['/p/CamCGL3LSHX/',
 '/p/Cag862TrWuX/',
 '/p/CaWWan2BoZc/',
 '/p/CaRgX_svAjY/',
 '/p/CaG5pIbv7MV/',
 '/p/CZ9rGIHP423/',
 '/p/CZx-sLRPoq_/',
 '/p/CZs1BDAqmYF/',
 '/p/CZmf9rhPnGY/',
 '/p/CZjB2LzP7mn/',
 '/p/CZMv78CvpIk/',
 '/p/CZL99vIDzxe/']

In [19]:
link_list = []
num_of_post = len(post_link)
for i in (range(num_of_post)):
    link_list.append("https://www.instagram.com" + post_link[i])
link_list

['https://www.instagram.com/p/CamCGL3LSHX/',
 'https://www.instagram.com/p/Cag862TrWuX/',
 'https://www.instagram.com/p/CaWWan2BoZc/',
 'https://www.instagram.com/p/CaRgX_svAjY/',
 'https://www.instagram.com/p/CaG5pIbv7MV/',
 'https://www.instagram.com/p/CZ9rGIHP423/']

In [29]:
for i in range(1):
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    print(soup.title.text)
    time.sleep(0.5)

Instagram의 gyeonggi_korea(경기도)님: “A complete guide to unique cafés in Yongin City of Gyeonggi Province

📢Real Garden K
Feel like having a ‘forest and phytoncide’ day?…”


In [37]:
a_page = bs(driver.page_source, 'lxml')
like_data = a_page.find('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
like_data.text

'좋아요 24개'

In [40]:
type(like_data)

bs4.element.Tag

In [44]:
num_of_like = int(like_data.text[4:-1])
num_of_like

24

In [46]:
for i in range(1,2):
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    print(soup.title.text)
    time.sleep(0.5)

Instagram의 gyeonggi_korea(경기도)님: “Mystical constellations and planets🌎,
Don’t you want to see them all?
Let’s visit the Anseong Astronomical Science Museum! 🌃

Observe the…”


In [47]:
a_page = bs(driver.page_source, 'lxml')
like_data = a_page.find('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
like_data.text

'좋아요 24개'

In [48]:
for i in range(3):
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    print(soup.title.text)
    time.sleep(0.5)

    a_page = bs(driver.page_source, 'lxml')
    like_data = a_page.find('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
    num_of_like = int(like_data.text[4:-1])
    print(num_of_like)

Instagram의 gyeonggi_korea(경기도)님: “A complete guide to unique cafés in Yongin City of Gyeonggi Province

📢Real Garden K
Feel like having a ‘forest and phytoncide’ day?…”
24
Instagram의 gyeonggi_korea(경기도)님: “Mystical constellations and planets🌎,
Don’t you want to see them all?
Let’s visit the Anseong Astronomical Science Museum! 🌃

Observe the…”
24
Instagram의 gyeonggi_korea(경기도)님: “Shall we take a day trip
to nature-friendly Uiwang City of Gyeonggi Province? 🚓

📢Cheonggyesa🙏

A Buddhist temple on picturesque…”
26


# insta 좋아요 수 크롤링

In [94]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
import getpass

In [102]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.instagram.com/accounts/login/")

username = 'gyeonggi_korea'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
element_id.send_keys(username)
element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
element_password.send_keys(password)

time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

<ipython-input-102-9816f8534c9c>:7: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  element_id = driver.find_element_by_name("username")
<ipython-input-102-9816f8534c9c>:9: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  element_password = driver.find_element_by_name("password")
<ipython-input-102-9816f8534c9c>:15: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()


In [103]:
url = 'https://www.instagram.com/gyeonggi_korea/'
# webdriver 로드
driver.get(url)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title

<title>gyeonggi_korea(경기도)(@gyeonggi_korea) • Instagram 사진 및 동영상</title>

In [104]:
post_link = []
div = soup.find_all('div', class_="Nnq7C weEfm")

# 게시글 줄 순서(1번째부터 몇 번째 줄까지 가져올 것인지)
number_of_line = 2
# 게시글 링크 3개씩 가져오기
for row in range(number_of_line):
    a_tags = div[row].find_all('a')
    for i in range(3):
        link = a_tags[i].attrs['href']
        post_link.append(link)
# 수집한 링크 개수 확인
print("수집한 링크의 개수 : ", len(post_link))
print("수집한 링크 주소")
print(post_link)

In [105]:
title_list = []
like_list = []
for i in range(3):
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    temp_title = soup.title.text[34:]
    title_list.append(temp_title)
    print(temp_title)
    time.sleep(0.5)

    a_page = bs(driver.page_source, 'lxml')
    like_data = a_page.find('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
    num_of_like = int(like_data.text[4:-1])

    like_list.append(num_of_like)
    print(num_of_like)

A complete guide to unique cafés in Yongin City of Gyeonggi Province

📢Real Garden K
Feel like having a ‘forest and phytoncide’ day?…”
25
Mystical constellations and planets🌎,
Don’t you want to see them all?
Let’s visit the Anseong Astronomical Science Museum! 🌃

Observe the…”
24
Shall we take a day trip
to nature-friendly Uiwang City of Gyeonggi Province? 🚓

📢Cheonggyesa🙏

A Buddhist temple on picturesque…”
26


In [106]:
import pandas as pd
from datetime import datetime

In [107]:
insta_df = {'좋아요_수':num_of_like, '제목':title_list}
data = pd.DataFrame(insta_df)
data

,좋아요_수,제목
0,26,A complete guide to unique cafés in Yongin Cit...
1,26,"Mystical constellations and planets🌎,\nDon’t y..."
2,26,Shall we take a day trip\nto nature-friendly U...


In [108]:
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'insta_' + file_name + '.csv'
path = './crawling_data/'
data.to_csv(path+file_name, index=False, encoding='utf-8')

In [109]:
url = 'https://business.facebook.com/creatorstudio/instagram_content_posts'
# webdriver 로드
driver.get(url)

In [ ]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')